# load data and build a network

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)
bsize = 64
train_dataloader = DataLoader(training_data, batch_size=bsize)
test_dataloader = DataLoader(test_data, batch_size=bsize)

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()



# setting hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# add  loss function and optimizers

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define train_loop and test_loop

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

init the loss function and pass the data the train_loop and test_loop

In [5]:
epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------------
loss: 2.303370 [    0/60000]
loss: 2.287602 [ 6400/60000]
loss: 2.273006 [12800/60000]
loss: 2.271016 [19200/60000]
loss: 2.251667 [25600/60000]
loss: 2.248098 [32000/60000]
loss: 2.231467 [38400/60000]
loss: 2.210027 [44800/60000]
loss: 2.219783 [51200/60000]
loss: 2.204327 [57600/60000]
test error: 
 Accuracy: 59.0%, Avg loss: 0.034396 

Epoch 2
----------------------------------------
loss: 2.191906 [    0/60000]
loss: 2.180615 [ 6400/60000]
loss: 2.146415 [12800/60000]
loss: 2.164916 [19200/60000]
loss: 2.109900 [25600/60000]
loss: 2.119014 [32000/60000]
loss: 2.077165 [38400/60000]
loss: 2.040626 [44800/60000]
loss: 2.071718 [51200/60000]
loss: 2.042651 [57600/60000]
test error: 
 Accuracy: 59.8%, Avg loss: 0.031589 

Epoch 3
----------------------------------------
loss: 2.014366 [    0/60000]
loss: 1.989833 [ 6400/60000]
loss: 1.921960 [12800/60000]
loss: 1.973533 [19200/60000]
loss: 1.853410 [25600/60000]
loss: 1.900511 [32000/60

In [10]:
torch.save(model.state_dict(), "data/model_1.pth")
torch.save(model, "data/model_all.pth")

In [9]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Save and load model


In [ ]:
import torch.onnx as onnx
import torchvision.models as models


# Saving and loading only model weights

In [ ]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'data/model_weights.pth')

In [ ]:
model = models.vgg16()
model.load_state_dict(torch.load("data/model_weights.pth"))
model.eval()

> **Note:** Be sure to call `model.eval()` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.


# saving and loading models with shapes


In [ ]:
torch.save(model, "data/vgg_model.pth")
model = torch.load("data/vgg_model.pth")



## Exporting the model to ONNX

PyTorch also has native ONNX export support. Given the dynamic nature of the PyTorch execution graph, however, the export process must traverse the execution graph to produce a persisted ONNX model. For this reason, a test variable of the appropriate size should be passed in to the export routine (in our case, we will create a dummy zero tensor of the correct size):

In [ ]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'data/model.onnx')

In [6]:
import time
import numpy as np
np.random.seed(42)
a = np.random.uniform(size=(300, 300))
runtimes = 10

timecosts = []
for _ in range(runtimes):
    s_time = time.time()
    for i in range(100):
        a += 1
        np.linalg.svd(a)
    timecosts.append(time.time() - s_time)

print(f'mean of {runtimes} runs: {np.mean(timecosts):.5f}s')

mean of 10 runs: 1.74857s
